In [2]:
import tools
print(dir(tools))

['LineString', 'Moran', 'Polygon', 'Voronoi', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'box', 'build_g_borders', 'build_g_region', 'build_g_rook', 'coo_array', 'create_corrupted_graphs', 'csr_matrix', 'diags', 'extract_subset', 'generate_hex_lattice', 'generate_pent_lattice', 'generate_square_lattice', 'gpd', 'graph', 'identity', 'math', 'np', 'pd', 'plot_graph', 'plot_lattice', 'plt', 'polygonize', 'reindex', 'remove_random_edges', 'simulate_autocorrelated_data', 'sns', 'spmatrix', 'spsolve', 'triu', 'unary_union']


In [1]:
#from scipy.sparse import identity, csr_matrix
from scipy.sparse.linalg import spsolve
import geopandas as gpd
#import pandas as pd
import numpy as np
#from libpysal.weights import Queen, Rook
import matplotlib.pyplot as plt
from esda.moran import Moran
#from shapely.geometry import Polygon
#import copy
import seaborn as sns
from libpysal import graph,weights
#from splot.libpysal import plot_spatial_weights
#from scipy.sparse import spmatrix, triu, diags, coo_array
from tools import generate_square_lattice,remove_random_edges,simulate_autocorrelated_data
import itertools
from tqdm import tqdm
import pandas as pd

#from spreg import ML_Error

In [3]:
shapes = ['square','pent','hex']
sizes = [25,100,400]
corruption_methods = ['random','border','periphery','center']
n_runs = 10
c_runs = 10
rhos = np.arange(-0.9, 1.0, 0.1)
perc_missing = np.linspace(5,95, 19)  


In [ ]:
# 2. Calculate the total number of iterations (strictly for the progress bar to work)
total_iters = len(shapes) * len(sizes) * len(corruption_methods) * n_runs * c_runs * len(rhos) * len(perc_missing)

# 3. Create the combinations generator
# Note: Because n_runs and c_runs are integers, we wrap them in range()
combinations = itertools.product(
    shapes, 
    sizes, 
    corruption_methods, 
    range(n_runs), 
    range(c_runs), 
    rhos, 
    perc_missing
)

# 1. The Golden Rule: Create an empty list OUTSIDE the loop
all_results = []

for shape, size, method, n_run, c_run, rho, p in tqdm(combinations, total=total_iters, desc="Calculating Moran's I"):
    
    
    gdf = gpd.read_parquet(f'data\gdf_{shape}_{size}.parquet')
    wm = graph.read_parquet(f"graphs/{shape}/size_{size}/{method}/g_{int(p)}_{c_run}.parquet").to_W()
    col_prefix = f"rho_{rho:.1f}"
    y = gdf[f"{col_prefix}_run_{n_run}"].values
    mi_cor = Moran(y, wm)
    
    
    # 2. Build your dictionary using the exact loop variables
    iteration_result = {
        "shape": shape,
        "size": size,
        "corruption_method": method,
        "rho": rho,
        "p_missing": p,                
        "corruption_run": c_run,
        "data_run": n_run,
        "moran_i": mi_cor.I,
        "p_value": mi_cor.p_sim
    }
    
    # 3. Append the dictionary to your list (Python does this in nanoseconds)
    all_results.append(iteration_result)

# 4. OUTSIDE THE LOOP: Convert the 1.3 million dictionaries into a DataFrame instantly!
final_df = pd.DataFrame(all_results)

<>:22: SyntaxWarning: invalid escape sequence '\g'
<>:22: SyntaxWarning: invalid escape sequence '\g'
C:\Users\elapo\AppData\Local\Temp\ipykernel_42472\1158676420.py:22: SyntaxWarning: invalid escape sequence '\g'
  gdf = gpd.read_parquet(f'data\gdf_{shape}_{size}.parquet')
Calculating Moran's I:   0%|          | 8/1299600 [00:00<25:16:43, 14.28it/s]
C:\Users\elapo\AppData\Local\Temp\ipykernel_42472\1158676420.py:22: SyntaxWarning: invalid escape sequence '\g'
  gdf = gpd.read_parquet(f'data\gdf_{shape}_{size}.parquet')


FileNotFoundError: graphs/square/size_25/random/g_45_0.parquet